# Embed with swin and learn with xgboost !

Will it work ?

In [59]:
import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy

from transformers import SwinConfig, SwinForImageClassification, ViTImageProcessor, Trainer, TrainingArguments

import torch
from torchvision.io import read_image
from torch.utils.data import Dataset, random_split, DataLoader, WeightedRandomSampler
from torch.nn import CrossEntropyLoss
from torchmetrics.classification import MulticlassConfusionMatrix
from torchvision import transforms
from torchvision.datasets import ImageFolder
import gc

import xgboost

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
home = os.path.join(os.getcwd(), "..")
config_json = os.path.join(home, "data", "model", "config", "config.json")
preprocessor_config_json = os.path.join(home, "data", "model", "config", "preprocessor_config.json")
input_train = os.path.join(home, "data", "input", "Train")
model_path = os.path.join(home, "data", "model")

In [3]:
pretrained_model = SwinForImageClassification.from_pretrained("microsoft/swin-base-patch4-window7-224-in22k")

In [39]:
embedder = torch.nn.Sequential(*(list(pretrained_model.children())[:-1]))
# Set the model to evaluation mode
embedder.eval()

Sequential(
  (0): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOutput(
                  

In [13]:
with open(preprocessor_config_json, 'r') as f:
  vit_prepro_config_args = json.load(f)

In [15]:
extractor = ViTImageProcessor(**vit_prepro_config_args)

In [19]:
def parse_fn(filename):
    # <BIOPSY_PROCEDURE>_<TUMOR_CLASS>_<TUMOR_TYPE>-<YEAR>-<SLIDE_ID>-<MAG>-<SEQ>
    parsed = filename[:-4].replace('-', '_').split('_')
    parsed.append(filename)
    return parsed

def parse_type_id(filename):
    parsed = parse_fn(filename)
    label_maps = {'F':'1', 'DC':'2', 'PC':'3', 'PT':'4', 'MC':'5', 'LC':'6', 'A':'7', 'TA':'8'}
    return int(label_maps[parsed[2]])-1

In [20]:
class HistoDataset(Dataset):
    def __init__(self, img_dir, train=False, predict=False):
        self.img_dir = img_dir
        self.train = train
        self.predict = predict

    def __len__(self):
        return len(os.listdir(path=self.img_dir))

    def __getitem__(self, idx):
        filename = os.listdir(path=self.img_dir)[idx]
        img_path = os.path.join(self.img_dir, filename)
        image = read_image(img_path)
        if self.train:
            composed = transforms.Compose([transforms.ToPILImage(),
                                transforms.ColorJitter(brightness=.1, contrast=0, saturation=.1, hue=.1), 
                                transforms.RandomHorizontalFlip(0.3), 
                                transforms.RandomVerticalFlip(0.3),
                                transforms.RandomRotation(30),
                                transforms.TrivialAugmentWide(),
                                transforms.RandomApply(transforms=[transforms.RandomResizedCrop(size=(460, 700))], p=0.5),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=extractor.image_mean, std=extractor.image_std), 
                                transforms.RandomErasing(0.5),
                                ])
            image = composed(image)
        if self.predict:
            label='Unknown'
        else:
            label = parse_type_id(filename)
        image_features = extractor(image, return_tensors="pt").pixel_values.squeeze(0)
        return {'pixel_values':image_features, 'label':label, 'filename':filename}

In [21]:
histoDataset_train = HistoDataset(input_train, train=True)
train_dataset, _ = random_split(histoDataset_train, [0.78, 0.22], generator=torch.Generator().manual_seed(42))

histoDataset_eval_chal = HistoDataset(input_train, train=False)
_, eval_dataset = random_split(histoDataset_eval_chal, [0.78, 0.22], generator=torch.Generator().manual_seed(42))

In [25]:
train_dataset[0]['pixel_values'].shape

torch.Size([3, 224, 224])

In [50]:
features = []
for i, data in enumerate(train_dataset):
    inputs = extractor(data['pixel_values'], return_tensors="pt")
    with torch.no_grad():
        output = embedder(inputs['pixel_values']).pooler_output.numpy()
        features.append(output)

In [53]:
labels = []
for data in enumerate(train_dataset):
    labels.append(data['label'])

In [52]:
X_train = np.squeeze(np.array(features), axis=1)
X_train.shape

(330, 1024)

In [57]:
y_train = np.array(labels)
y_train.shape

(330,)

In [60]:
# Train an XGBoost classifier on the flattened feature vectors
clf = xgboost.XGBClassifier()
clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

## Prediction time

In [62]:
wd = os.path.join(os.getcwd(), '..')
data = os.path.join(wd, 'data', 'input')
image_test_path = os.path.join(data, 'Test')

# Test images
images_test = os.listdir(path=image_test_path)

def parseTest_fn(filename):
    # <BIOPSY_PROCEDURE>_<ID>
    parsed = filename[:-4].split('_')
    parsed.append(filename)
    return parsed

columns = ['procedure', 'id', 'filename']

df_test = pd.DataFrame(list(map(parseTest_fn, images_test)), columns=columns)
df_test['type_id'] = 0
df_test.head()

,procedure,id,filename,type_id
0,SOB,1,SOB_1.png,0
1,SOB,10,SOB_10.png,0
2,SOB,100,SOB_100.png,0
3,SOB,101,SOB_101.png,0
4,SOB,102,SOB_102.png,0


In [72]:
# Test images
data = os.path.join(wd, 'data', 'input')
image_test_path = os.path.join(data, 'Test')
submission_path = os.path.join(wd, 'data', 'output', 'submission', 'pred_swin_xgboost_20230306_2332.csv')
images_test = os.listdir(image_test_path)


# Number of images
print("Number of images for the test set: ", len(images_test))

Number of images for the test set:  207


In [73]:
def parseTest_fn(filename):
    # <BIOPSY_PROCEDURE>_<ID>
    parsed = filename[:-4].split('_')
    parsed.append(filename)
    return parsed

columns = ['procedure', 'id', 'filename']

df_test = pd.DataFrame(list(map(parseTest_fn, images_test)), columns=columns)
print(df_test.head())
print(df_test.shape)

  procedure   id     filename
0       SOB    1    SOB_1.png
1       SOB   10   SOB_10.png
2       SOB  100  SOB_100.png
3       SOB  101  SOB_101.png
4       SOB  102  SOB_102.png
(207, 3)


In [74]:
test_dataset = HistoDataset(image_test_path, train=False, predict=True)

In [75]:
features_test = []
for i, data in enumerate(test_dataset):
    inputs = extractor(data['pixel_values'], return_tensors="pt")
    with torch.no_grad():
        output_test = embedder(inputs['pixel_values']).pooler_output.numpy()
        features_test.append(output_test)

In [76]:
X_test = np.squeeze(np.array(features_test), axis=1)
X_test.shape

(207, 1024)

In [77]:
y_pred_test = clf.predict(X_test)+1
df_pred = pd.concat([df_test['id'], pd.Series(y_pred_test, name='type_id')], axis=1)
display(df_pred['type_id'].value_counts())
df_pred.to_csv(submission_path, index=False, header=False)

7    108
2     37
5     25
1     21
3     12
4      4
Name: type_id, dtype: int64

It does not work !